# Kernel ya Semantiki

Katika mfano huu wa msimbo, utatumia Mfumo wa AI wa [Kernel ya Semantiki](https://aka.ms/ai-agents-beginners/semantic-kernel) kuunda wakala wa msingi.

Lengo la mfano huu ni kukuonyesha hatua ambazo baadaye tutatumia katika mifano mingine ya msimbo tunapotekeleza mifumo tofauti ya kiwakala.


## Ingiza Vifurushi vya Python Vinavyohitajika


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Kuunda Mteja

Katika mfano huu, tutatumia [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) kwa ajili ya kupata LLM.

`ai_model_id` imefafanuliwa kama `gpt-4o-mini`. Jaribu kubadilisha mfano huu na kutumia modeli nyingine inayopatikana kwenye soko la GitHub Models ili kuona matokeo tofauti.

Ili tuweze kutumia `Azure Inference SDK` ambayo inatumika kwa `base_url` ya GitHub Models, tutatumia kiunganishi cha `OpenAIChatCompletion` ndani ya Semantic Kernel. Pia kuna [viunganishi vingine vinavyopatikana](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) vya kutumia Semantic Kernel kwa watoa huduma wengine wa modeli.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Kuunda Wakala

Hapa tunaunda Wakala anayeitwa `TravelAgent`.

Kwa mfano huu, tunatumia maagizo rahisi sana. Unaweza kubadilisha maagizo haya ili kuona jinsi wakala anavyojibu kwa njia tofauti.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Kuendesha Wakala

Sasa tunaweza kuendesha Wakala kwa kufafanua thread ya aina `ChatHistoryAgentThread`. Ujumbe wowote wa mfumo unaohitajika hutolewa kwa neno la msingi `messages` la invoke_stream ya wakala.

Baada ya haya kufafanuliwa, tunaunda `user_inputs` ambayo itakuwa kile mtumiaji anachotuma kwa wakala. Katika hali hii, tumeweka ujumbe huu kuwa `Plan me a sunny vacation`.

Jisikie huru kubadilisha ujumbe huu ili kuona jinsi wakala anavyojibu kwa njia tofauti.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya kutafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kuhakikisha usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, tafsiri ya kitaalamu ya binadamu inapendekezwa. Hatutawajibika kwa kutoelewana au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
